### file 읽어오기

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import TextLoader
path = r'data/medicine.txt'

# 1. loader 객체 생성
loader = TextLoader(path, encoding='utf-8')
# 2. loader를 이용해서 파일 읽어옴
docs = loader.load()

In [3]:
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.globals import set_llm_cache
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

store = {}
config = {"configurable": {"session_id":"chat_message_1"}}

set_llm_cache(SQLiteCache(".cache_prompt.sqlite"))

COLLECTION_NAME = "medicine_docs"
PERSIST_DIRECTORY = "vector_store/chroma/medicine_db7"
EMBEDDING_MODEL_NAME = 'text-embedding-3-small'
from langchain_openai import OpenAIEmbeddings

# Text Loading
loader = TextLoader(path, encoding='utf-8')

# 문서 load and split
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

docs = loader.load_and_split(splitter)

# Vector Store 생성

embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

# Retriever 생성 - "mmr" 방식
# MMR 방식 조회
retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs = {
        'k':5,
        'fetch_k':10
    }
)



In [4]:
store = {}
config = {"configurable": {"session_id":"chat_message_1"}}

def get_session_history(session_id):
    
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]
    
    memory = ConversationBufferWindowMemory(
        chat_memory=store[session_id], # 메세지 저장소 추가.
        k=2,
        return_messages=True,
        message_key="history"
    )
    # 1. memory에서 저장된 message들을 조회
    message_list = memory.load_memory_variables({})['history']# list[Message]
    # 2. 조회한 message들을 InMemoryChatMessageHistory에 추가
    store[session_id] = InMemoryChatMessageHistory(messages=message_list)
    # 3. 2의 InMemoryChatMessageHistory를 반환.
    return store[session_id]

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from langchain.memory import ConversationBufferWindowMemory
from operator import itemgetter
from langchain_openai import ChatOpenAI

# Prompt Template 생성
messages = [
            ("ai", """
            너는 유능한 약사야. context 안에 있는 내용에 한 해서
            어떤 효과가 있는지 알려주고 사용자가 입력한 증상에 맞는 약을 5개만 선택해서 알려줘.
            복용이 간단하고 부작용이 적은 약을 우선적으로 추천해서 알려줘.
            약을 복용하기 전에 반드시 알아야 할 사항은 꼭 알려줘.
            증상이 악화될 경우 대처방법에 대해서 알려줘.
            약에 따른 복용법에 대해서도 간단히 언급해줘.
            주의사항 요약해서 알려줘. 
            위의 내용을 표로 정리해서 알려줘.
            사용자가 말한 증상을 효능 부분에서 찾아서 그에 맞는 약을 알려줘.     
            {context}")"""),
            MessagesPlaceholder("history"), 
            ("human", "{question}")
    ]

prompt_template = ChatPromptTemplate(messages)

model = ChatOpenAI(model='gpt-4o-mini')

parser = StrOutputParser()

def get_context(input_data:dict):
    return retriever.invoke(input_data['question'])

# Chain 구성 retriever (관련문서 조회) -> prompt_template -> model -> outputparser
runnable = {'context':RunnableLambda(get_context), 'question':itemgetter("question"), 'history':itemgetter('history')} | prompt_template | model | parser


chain = RunnableWithMessageHistory(
    runnable=runnable,
    get_session_history=get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

config = {"configurable": {"session_id":"id-1"}}
data = chain.invoke({"question":"두통인데 약 추천해줘"}, config)

print(data)


두통에 효과적인 약을 다음과 같이 추천드립니다. 아래는 각 약물의 효능, 복용법, 주의사항과 함께 정리된 표입니다.

| 약물명         | 효능 및 효과                                            | 복용법                                   | 주의사항                                              |
|----------------|------------------------------------------------------|----------------------------------------|----------------------------------------------------|
| 속콜펜정       | 두통, 치통, 발치 후 동통, 인후통, 귀의 통증 등 진통 | 만 15세 이상: 1회 1정, 1일 3회, 4시간 간격 | 과민증 환자, 알레르기 체질, 임신 가능성 있는 여성은 사용 전 상담 필요 |
| 이부플러스생정 | 두통, 치통, 근육통, 생리통 등 진통 및 해열           | 만 15세 이상: 1회 2정, 1일 3회, 4시간 간격 | 과민증 환자, 임신 가능성 있는 여성은 사용 전 상담 필요 |
| 타나센정       | 두통, 신경통, 생리통, 외상으로 인한 동통 등 진통    | 성인: 6시간마다 2정, 1일 최대 8정   | 소화성 궤양, 심부전 환자는 사용 전 상담 필요      |
| 그날엔정       | 두통, 치통, 근육통, 생리통 등 진통 및 해열           | 15세 이상: 1회 2정, 1일 3회, 4시간 간격 | 과민증 환자, 소화성 궤양, 심부전 환자는 사용 전 상담 필요 |
| 더마큐어로션   | 두통과는 관련 없으나 피부 가려움증 및 습진에 사용    | 1일 1~3회 환부에 바름                 | 피부 감염증이 있는 경우 사용 금지, 임신 및 수유 시 상담 필요 |

### 복용

In [6]:
data = chain.invoke({"question":"첫번째 약에 대해서 더 설명해줘"}, config)

print(data)

/var/folders/8y/jsfncy6n39v2jylvdpm61nmh0000gn/T/ipykernel_35880/1752197439.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


속콜펜정에 대한 자세한 설명은 다음과 같습니다.

### 속콜펜정

**효능 및 효과:**
- 속콜펜정은 두통, 치통, 발치 후 동통, 인후통, 귀의 통증 등 다양한 통증을 완화하는 데 사용됩니다. 특히, 일반적인 두통 및 긴장성 두통에 효과적입니다.

**주 성분:**
- 주성분은 아세트아미노펜과 같은 진통제입니다. 이 성분은 뇌의 통증 신호를 차단하고, 열을 낮추는 작용을 합니다.

**복용법:**
- 성인(만 15세 이상): 1회 1정, 1일 3회 복용하며, 복용 간격은 최소 4시간 이상 두어야 합니다.
- 필요에 따라 의사의 지시에 따라 복용량을 조정할 수 있습니다.

**복용 시 주의사항:**
- **과민증:** 아세트아미노펜에 과민증이 있는 환자는 복용을 피해야 합니다.
- **간 질환:** 간 장애가 있는 환자, 특히 알코올을 자주 섭취하는 사람은 사용 시 주의해야 합니다.
- **임신 및 수유:** 임신 중이거나 수유 중인 여성은 의사와 상담 후 사용해야 합니다.
- **장기 복용:** 장기간 복용하지 말고, 5~6일 이상 복용 후에도 증상이 개선되지 않으면 의사와 상담해야 합니다.

**부작용:**
- 일반적으로 부작용이 적지만, 드물게 간 손상, 피부 발진, 알레르기 반응 등이 발생할 수 있습니다. 이러한 증상이 나타나면 즉시 복용을 중단하고 의사와 상담해야 합니다.

**보관 방법:**
- 어린이의 손이 닿지 않는 곳에 보관하고, 습기와 빛을 피해 실온에서 보관해야 합니다.

속콜펜정은 일반적으로 안전성이 높은 약물로, 두통 완화에 효과적입니다. 그러나 복용 전에 자신의 건강 상태와 병력에 대해 충분히 고려하고, 필요할 경우 전문의와 상담하는 것이 좋습니다.
